In [ ]:
# Required (qiskit)
!pip install qiskit
!pip install qiskit_aer

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt


In [ ]:
%matplotlib inline

def run_bb84_simulation(num_bits):
    alice_bits = np.random.choice([0, 1], size=num_bits)
    alice_bases = np.random.choice([0, 1], size=num_bits)
    bob_bases = np.random.choice([0, 1], size=num_bits)

    qc = QuantumCircuit(num_bits, num_bits)
    for i in range(num_bits):
        if alice_bases[i] == 1:
            qc.h(i)
        if alice_bits[i] == 1:
            qc.x(i)
        if alice_bases[i] == 1:
            qc.h(i)
    qc.barrier()
    for i in range(num_bits):
        if bob_bases[i] == 1:
            qc.h(i)
        qc.measure(i, i)

    simulator = AerSimulator()
    job = simulator.run(qc, shots=4000)
    result = job.result()
    measurements = result.get_counts()
    if isinstance(measurements, dict):
        plot_histogram(measurements, title="BB84 Measurement Outcomes")
        display(plot_histogram(measurements))
    else:
        print("Measurements are not in the expected dictionary format.")
    plot_histogram(result.get_counts(), title="BB84 Measurement Outcomes")
    measured_bits = list(measurements.keys())[0][::-1]
    sifted_key = [(alice_bits[i], int(measured_bits[i])) for i in range(num_bits) if alice_bases[i] == bob_bases[i]]
    qber = calculate_qber(sifted_key)
    final_key = None
    if qber > 0.25:
        print("QBER too high, aborting protocol.")
    else:
        final_key = privacy_amplification(sifted_key)

    return alice_bits, alice_bases, bob_bases, measurements, measured_bits, qber, sifted_key, final_key

def calculate_qber(sifted_key):
    errors = sum(1 for a, b in sifted_key if a != b)
    return errors / len(sifted_key) if sifted_key else 0

def privacy_amplification(sifted_key):
    from hashlib import sha256
    key_string = ''.join(str(bit) for _, bit in sifted_key)
    hashed_key = sha256(key_string.encode()).hexdigest()
    return hashed_key




Running the Protocol

In [ ]:
%matplotlib inline

num_bits = 10
alice_bits, alice_bases, bob_bases, measurements,measured_basis, qber, sifted_key, final_key = run_bb84_simulation(num_bits)


In [ ]:
counts = measurements
shots = sum(counts.values())
probabilities = {k: v / shots for k, v in counts.items()}

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(range(len(probabilities)), probabilities.values())
ax.set_xlabel('States')
ax.set_ylabel('Probabilities')
ax.set_title('BB84 Probabilities')
ax.set_xticks(range(len(probabilities)))
ax.set_xticklabels(probabilities.keys(), rotation=90)
plt.show()


